In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Acquisition
Flat acquisition, i dati sono già disponibili online da: grouplens.org

In [9]:
# Flat acquisition from csv files
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')
genome_score = pd.read_csv('data/genome-scores.csv')
genome_tags = pd.read_csv('data/genome-tags.csv')

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
   userId  movieId               tag   timestamp
0       3      260           classic  1439472355
1       3      260            sci-fi  1439472256
2       4     1732       dark comedy  1573943598
3       4     1732    great dialogue  1573943604
4       4     7569  so bad it's good  1573943455


In [13]:
# Displaying the first 5 rows of each dataset
print("Movies.csv")
display(movies.head())
print("Ratings.csv")
display(ratings.head())
print("Tags.csv")
display(tags.head())
print("Genome-scores.csv")
display(genome_score.head())
print("Genome-tags.csv")
display(genome_tags.head())

Movies.csv


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Ratings.csv


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


Tags.csv


,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


Genome-scores.csv


,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


Genome-tags.csv


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


- <b>Movies:</b> contiene id, titolo e genere di ogni film
- <b>Ratings:</b> contiene tutti i ratings associati per ogni film da ogni utente
- <b>Tags:</b> contiene tutti i tag associati per ogni film da ogni utente
- <b>Genome-scores:</b> contiene per ogni tag associato ad un film la sua rilevanza
- <b>Genome-tags:</b> contiene per ogni tag il suo id e il suo nome

# Data Visualization
Proviamo a visualizzare i dati per capire meglio come sono strutturati